In [ ]:
import pandas as pd
import re
import yaml
import sqlparse
import os
import pandas as pd
import requests

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', 10) 

#### Read dbs